In [48]:
import pandas as pd

In [49]:
def inspect_csv(filepath, num_rows=5):
    """
    Reads a CSV file, prints the number of records, and displays the first few rows.

    Args:
        filepath: Path to the CSV file.
        num_rows: Number of rows to display (default: 5).
    """
    try:
        df = pd.read_csv(filepath, index_col=0)  # Read CSV, using first column as index
        print(f"File: {filepath}")
        print(f"Number of records: {len(df)}")
        print(f"First {num_rows} rows:\n")
        print(df.head(num_rows))
        print("-" * 40)  # Separator for better readability

    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
    except Exception as e:
        print(f"An error occurred while processing {filepath}: {e}")

In [50]:
file_paths = [
    'data/p_icd9NotesDataTable.csv',
    'data/p_icd9NotesDataTable_train.csv',
    'data/p_icd9NotesDataTable_valid.csv',
]

In [18]:
for path in file_paths:
    if ".txt" in path: #training_data.txt does not have any headers.
        try:
            with open(path, 'r') as file:
                print(f"File: {path}")
                # Count lines in the file
                line_count = 0
                first_lines = []
                for i, line in enumerate(file):
                    line_count += 1
                    if i < 5:  # store first 5 lines
                        first_lines.append(line.strip())
                print(f"Number of records: {line_count}")
                print(f"First 5 rows:\n")
                for l in first_lines:
                    print(l)

        except FileNotFoundError:
            print(f"Error: File not found at {path}")
        except Exception as e:
            print(f"An error occurred while processing {path}: {e}")
        print("-" * 40)  # Separator for better readability

    else:
        inspect_csv(path)

File: data/p_icd9NotesDataTable.csv
Number of records: 52722
First 5 rows:

         SUBJECT_ID                                          ICD9_CODE  \
HADM_ID                                                                  
100001        58526  25013-3371-5849-5780-V5867-25063-5363-4580-250...   
100003        54610  53100-2851-07054-5715-45621-53789-4019-53550-7823   
100006         9895   49320-51881-486-20300-2761-7850-3090-V1251-V1582   
100007        23018                           56081-5570-9973-486-4019   
100009          533  41401-99604-4142-25000-27800-V8535-4148-4111-V...   

          CHARTDATE DESCRIPTION  \
HADM_ID                           
100001   2117-09-17      Report   
100003   2150-04-21      Report   
100006   2108-04-17      Report   
100007   2145-04-07      Report   
100009   2162-05-21      Report   

                                                      TEXT  \
HADM_ID                                                      
100001   Admission Date:  [**2117-9

In [51]:
def compare_csv_files(file1_path, file2_path, key_columns, exclude_columns=None):
    """
    Compares two CSV files record by record, focusing on specified key columns
    to identify differences.  Lists records from both files sharing the same
    key values as the differing records.  Option to exclude columns from the comparison.

    Args:
        file1_path (str): Path to the first CSV file (R output).
        file2_path (str): Path to the second CSV file (Python output).
        key_columns (list): List of column names to use as keys for comparing records (e.g., ['HADM_ID', 'SUBJECT_ID']).
        exclude_columns (list, optional): List of column names to exclude from the comparison. Defaults to None.

    Returns:
        pandas.DataFrame: DataFrame containing records that are different between the two files.
                          Returns an empty DataFrame if no differences are found.  Also prints
                          related records.
    """

    try:
        # Read CSV files into pandas DataFrames
        df1 = pd.read_csv(file1_path)
        df2 = pd.read_csv(file2_path)

        # Print record counts before comparison
        print(f"Record count in {file1_path}: {len(df1)}")
        print(f"Record count in {file2_path}: {len(df2)}")

        # Ensure key columns exist in both DataFrames
        for col in key_columns:
            if col not in df1.columns or col not in df2.columns:
                raise ValueError(f"Key column '{col}' not found in one or both files.")

        # Exclude specified columns
        if exclude_columns:
            for col in exclude_columns:
                if col in df1.columns:
                    df1 = df1.drop(col, axis=1)
                if col in df2.columns:
                    df2 = df2.drop(col, axis=1)

        # Convert DataFrames to string format before merging and comparison
        df1 = df1.astype(str)
        df2 = df2.astype(str)

        # Merge the DataFrames using the specified key columns as the join key
        merged_df = pd.merge(df1, df2, on=key_columns, how='outer', indicator=True)

        # Filter for records that are present in only one of the DataFrames
        different_records = merged_df[merged_df['_merge'] != 'both']

        if not different_records.empty:
            print("\nSummary of Differences:")
            print(different_records['_merge'].value_counts())

            print("\nDifferent Records:")
            print(different_records)

            # Identify keys (HADM_ID, SUBJECT_ID) of different records from Python output
            python_extra_keys = different_records[different_records['_merge'] == 'right_only'][key_columns]

            # List records from Python file that share those keys
            if not python_extra_keys.empty:
                print("\nRecords from Python file sharing keys with extra records:")
                for index, row in python_extra_keys.iterrows():
                    # Build query string
                    query_parts = []
                    for key_col in key_columns:
                        query_parts.append(f"`{key_col}` == '{row[key_col]}'")  # Escape column names
                    query = ' and '.join(query_parts)
                    print(f"\nRecords in Python file with {key_columns} == {row.to_dict()}:")
                    print(df2.query(query))

            # Identify keys (HADM_ID, SUBJECT_ID) of different records from R output
            r_extra_keys = different_records[different_records['_merge'] == 'left_only'][key_columns]

            # List records from R file that share those keys
            if not r_extra_keys.empty:
                print("\nRecords from R file sharing keys with extra records:")
                for index, row in r_extra_keys.iterrows():
                    # Build query string
                    query_parts = []
                    for key_col in key_columns:
                        query_parts.append(f"`{key_col}` == '{row[key_col]}'")  # Escape column names
                    query = ' and '.join(query_parts)
                    print(f"\nRecords in R file with {key_columns} == {row.to_dict()}:")
                    print(df1.query(query))
        else:
            print("\nNo differences found between the files.")

        return different_records  # Return the DataFrame containing different records

    except FileNotFoundError:
        print("Error: One or both of the specified files were not found.")
        return pd.DataFrame()  # Return an empty DataFrame in case of an error
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

In [52]:
# Specify the paths to the CSV files you want to compare
file1_path = 'data/icd9NotesDataTable.csv'  # Path to the CSV file generated by the R script
file2_path = 'data/p_icd9NotesDataTable.csv'  # Path to the CSV file generated by the Python script

In [53]:
# Specify the key columns to use for comparing records (adjust as needed)
key_columns = ['HADM_ID', 'SUBJECT_ID']  # Columns that uniquely identify each record

# Specify the columns to exclude from the comparison
exclude_columns = ['TEXT']  # List of columns to exclude

# Compare the CSV files and get the DataFrame of different records
different_records_df = compare_csv_files(file1_path, file2_path, key_columns, exclude_columns)

# Optionally, save the DataFrame of different records to a CSV file
if not different_records_df.empty:
    different_records_df.to_csv('data/different_records.csv', index=False)
    print("\nDifferent records saved to 'data/different_records.csv'")
else:
    print("\nNo differences found between the files.")

Error: One or both of the specified files were not found.

No differences found between the files.


In [54]:
def count_records(file_path):
    """
    Counts the number of records (lines) in a text file.

    Args:
        file_path (str): Path to the text file.

    Returns:
        int: The number of records in the file.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            record_count = sum(1 for line in f)  # Efficiently count lines
        return record_count
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"An error occurred while counting records in '{file_path}': {e}")
        return None


# Define the paths to the text files
train_text_file = 'data/train.txt'
train_labels_file = 'data/train_labels.txt'
valid_text_file = 'data/valid.txt'
valid_labels_file = 'data/valid_labels.txt'

# Count the records in each file
train_text_count = count_records(train_text_file)
train_labels_count = count_records(train_labels_file)
valid_text_count = count_records(valid_text_file)
valid_labels_count = count_records(valid_labels_file)

# Print the results
if train_text_count is not None:
    print(f"Number of records in '{train_text_file}': {train_text_count}")
if train_labels_count is not None:
    print(f"Number of records in '{train_labels_file}': {train_labels_count}")
if valid_text_count is not None:
    print(f"Number of records in '{valid_text_file}': {valid_text_count}")
if valid_labels_count is not None:
    print(f"Number of records in '{valid_labels_file}': {valid_labels_count}")

# Verify if the counts match between text and labels files
if (train_text_count is not None and train_labels_count is not None and
    train_text_count != train_labels_count):
    print("Warning: Number of records in train.txt and train_labels.txt do not match!")

if (valid_text_count is not None and valid_labels_count is not None and
    valid_text_count != valid_labels_count):
    print("Warning: Number of records in valid.txt and valid_labels.txt do not match!")

Number of records in 'data/train.txt': 39541
Number of records in 'data/train_labels.txt': 39541
Number of records in 'data/valid.txt': 13181
Number of records in 'data/valid_labels.txt': 13181


In [88]:
import csv
from collections import defaultdict, Counter

def count_categories_by_subject(filename):
    """
    1. Reads 'filename' as a CSV.
    2. Gathers all unique 'TopLevelICD' categories (which may be comma-separated)
       for each SUBJECT_ID.
    3. Counts how many unique subjects have each category.
    """

    # Step 1: Accumulate categories by subject
    subject_categories = defaultdict(set)
    counter = 0
    
    with open(filename, 'r', newline='', encoding='utf-8') as csv_file:
        reader = csv.DictReader(csv_file)
        
        for row in reader:
            counter = counter + 1
            subject_id = int(row['HADM_ID'])
            #print(subject_id)
            #subject_id = counter
            
            # Ensure we handle potential multiple comma-separated categories
            category_string = row['TopLevelICD'].strip()
            if category_string:
                categories = [cat.strip() for cat in category_string.split('-')]
                # Add these categories to the subject's set
                for cat in categories:
                    cat_number = int(cat.split("cat:")[1])
                    subject_categories[subject_id].add(cat_number)

    # Step 2: Count how many subjects have each category
    category_counts = Counter()
    for subject_id, categories_set in subject_categories.items():
        for cat in categories_set:
            category_counts[cat] += 1

    # Step 3: Calculate total number of unique subjects
    total_subjects = len(subject_categories)
    print(f'Total groups: {total_subjects}')
    
    # Step 4: Print results (count and prevalence), sorted by category name
    print("Count of subjects per category (TopLevelICD) and prevalence (%), sorted by category:")
    for category in sorted(category_counts):
        count = category_counts[category]
        prevalence = (count / total_subjects) * 100
        print(f"{category}: {count} subjects ({prevalence:.2f}%)")

In [89]:
csv_filename = 'data/p_icd9NotesDataTable_train.csv'
count_categories_by_subject(csv_filename)

Total groups: 36905
Count of subjects per category (TopLevelICD) and prevalence (%), sorted by category:
1: 9957 subjects (26.98%)
2: 5960 subjects (16.15%)
3: 24198 subjects (65.57%)
4: 13332 subjects (36.13%)
5: 11006 subjects (29.82%)
6: 9355 subjects (25.35%)
7: 2597 subjects (7.04%)
8: 28931 subjects (78.39%)
9: 17202 subjects (46.61%)
10: 14333 subjects (38.84%)
11: 14817 subjects (40.15%)
12: 100 subjects (0.27%)
13: 4102 subjects (11.12%)
14: 6793 subjects (18.41%)
15: 1991 subjects (5.39%)
16: 2615 subjects (7.09%)
17: 13642 subjects (36.97%)
18: 15272 subjects (41.38%)
19: 11130 subjects (30.16%)
20: 20528 subjects (55.62%)


In [75]:
csv_filename = 'data/icd9NotesDataTable.csv'
count_categories_by_subject(csv_filename)

Total groups: 52722
Count of subjects per category (TopLevelICD) and prevalence (%), sorted by category:
1: 14212 subjects (26.96%)
2: 8579 subjects (16.27%)
3: 34600 subjects (65.63%)
4: 19006 subjects (36.05%)
5: 15731 subjects (29.84%)
6: 15377 subjects (29.17%)
7: 41335 subjects (78.40%)
8: 24577 subjects (46.62%)
9: 20430 subjects (38.75%)
10: 21253 subjects (40.31%)
11: 156 subjects (0.30%)
12: 5924 subjects (11.24%)
13: 9808 subjects (18.60%)
14: 2805 subjects (5.32%)
15: 3713 subjects (7.04%)
16: 19498 subjects (36.98%)
17: 21865 subjects (41.47%)
18: 15969 subjects (30.29%)
19: 29312 subjects (55.60%)


In [92]:
print(eval("1-15e-2"))

0.85
